<a href="https://colab.research.google.com/github/VGCH/video_translate/blob/main/video_translate_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchaudio soundfile librosa openai-whisper transformers sentencepiece moviepy sacremoses
!pip install -q silero num2words

In [ ]:
import torch
import whisper
from transformers import MarianMTModel, MarianTokenizer
import soundfile as sf
import librosa
import numpy as np
import os
import re
import subprocess
from google.colab import files
from num2words import num2words as num2words_lib
from silero import silero_tts

In [ ]:
# Инициализация устройств
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")

In [ ]:
# 1. Загрузка видеофайла в Colab
def upload_video_file():
    print("Пожалуйста, загрузите видеофайл:")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"Загружен файл: {filename}")
        return filename

    return None

In [ ]:
# 2. Извлечение аудио из видео
def extract_audio_from_video(video_path, audio_output='audio_original.wav'):
    try:
        # Используем moviepy для извлечения аудио
        from moviepy.editor import VideoFileClip

        print("Извлекаем аудио из видео...")
        video = VideoFileClip(video_path)
        audio = video.audio
        audio.write_audiofile(audio_output, fps=44100, verbose=False, logger=None)
        video.close()

        print(f"Аудио сохранено как: {audio_output}")
        return audio_output

    except Exception as e:
        print(f"Ошибка при извлечении аудио: {e}")
        # Пробуем через ffmpeg
        import subprocess
        try:
            subprocess.run([
                'ffmpeg', '-i', video_path, '-q:a', '0', '-map', 'a',
                audio_output, '-y'
            ], check=True, capture_output=True)
            return audio_output
        except:
            print("Не удалось извлечь аудио")
            return None

In [ ]:
# 3. Транскрибирование с временными метками
def transcribe_with_timestamps(audio_path):
    print("Загружаем модель Whisper...")
    model = whisper.load_model("medium")  # или "small", "medium", "large", "base"

    print("Транскрибируем аудио...")
    result = model.transcribe(audio_path, word_timestamps=True, language='en')

    segments = []
    for segment in result['segments']:
        segments.append({
            'start': segment['start'],
            'end': segment['end'],
            'text': segment['text'].strip(),
            'words': segment.get('words', [])
        })

    print(f"Транскрибировано {len(segments)} сегментов")
    return segments

In [ ]:
# 4. Переводчик
class Translator:
    def __init__(self):
        print("Загружаем модель перевода...")
        self.model_name = "Helsinki-NLP/opus-mt-en-ru"
        self.tokenizer = MarianTokenizer.from_pretrained(self.model_name)
        self.model = MarianMTModel.from_pretrained(self.model_name).to(device)

    def translate_text(self, text):
        # Очищаем текст от лишних пробелов
        text = ' '.join(text.split())
        if not text or len(text.strip()) == 0:
            return ""

        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(**inputs, max_length=512)

        translated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return translated

    def translate_segments(self, segments):
        print("Переводим сегменты...")
        translated_segments = []

        for i, segment in enumerate(segments):
            if not segment['text'].strip():
                continue

            print(f"Переводим сегмент {i+1}/{len(segments)}: {segment['text'][:50]}...")
            translated_text = self.translate_text(segment['text'])

            translated_segments.append({
                'start': segment['start'],
                'end': segment['end'],
                'original_text': segment['text'],
                'translated_text': translated_text,
                'duration': segment['end'] - segment['start']
            })

        return translated_segments

In [ ]:
# 5.  Синтез речи для перевода
class SileroTTSWithTiming:
    def __init__(self):
        print("Загружаем Silero TTS...")
        self.model, _ = silero_tts(language='ru', speaker='v5_ru')
        self.model.to(device)
        self.sample_rate = 48000
        print("Готов")

    def preprocess_text(self, text: str) -> str:
        """Предобработка текста с использованием num2words"""

        def convert_numbers(match):
            """Конвертирует числа в слова с учетом контекста"""
            num_str = match.group()

            try:
                # Пробуем преобразовать в число
                num = float(num_str) if '.' in num_str else int(num_str)

                # ОСОБЫЙ СЛУЧАЙ: ГОДА (4-значные числа)
                if len(num_str) == 4 and 1000 <= num <= 2099:
                    # Года произносим по цифрам: 2024 → "два ноль два четыре"
                    digits = {
                        '0': 'ноль', '1': 'один', '2': 'два', '3': 'три',
                        '4': 'четыре', '5': 'пять', '6': 'шесть', '7': 'семь',
                        '8': 'восемь', '9': 'девять'
                    }
                    return ' '.join(digits[digit] for digit in num_str)

                # ОСОБЫЙ СЛУЧАЙ: НОМЕРА ТЕЛЕФОНОВ, КОДЫ (6-12 цифр)
                if len(num_str) in [6, 7, 8, 10, 11, 12]:
                    # Произносим по цифрам
                    digits = {
                        '0': 'ноль', '1': 'один', '2': 'два', '3': 'три',
                        '4': 'четыре', '5': 'пять', '6': 'шесть', '7': 'семь',
                        '8': 'восемь', '9': 'девять'
                    }
                    return ' '.join(digits[digit] for digit in num_str)

                # ОБЫЧНЫЕ ЧИСЛА: используем num2words
                # Для дробных чисел
                if '.' in num_str:
                    whole_part = num2words_lib(int(num), lang='ru')
                    decimal_part = ' '.join(digits.get(d, d) for d in num_str.split('.')[1])
                    return f"{whole_part} целых {decimal_part}"

                # Для целых чисел
                return num2words_lib(num, lang='ru')

            except (ValueError, OverflowError):
                # Если не удалось преобразовать, возвращаем как есть
                return num_str

        # Шаг 1: Преобразуем год (особый случай)
        text = re.sub(r'\b(19|20)\d{2}\b', convert_numbers, text)

        # Шаг 2: Преобразуем номера телефонов (6-12 цифр подряд)
        text = re.sub(r'\b\d{6,12}\b', convert_numbers, text)

        # Шаг 3: Преобразуем обычные числа
        text = re.sub(r'\b\d+(?:\.\d+)?\b', convert_numbers, text)

        # Шаг 4: Обработка специальных символов и математических операций
        replacements = {
            '%': ' процент',
            '$': ' доллар',
            '€': ' евро',
            '₽': ' рубль',
            '£': ' фунт',
            '¥': ' иена',
            '+': ' плюс',
            '*': ' умножить на',
            '÷': ' разделить на',
            '=': ' равно',
            '/': ' на',
            '>': ' больше',
            '<': ' меньше'
        }

        for char, replacement in replacements.items():
            text = text.replace(char, replacement)

        # Шаг 5: Обработка процентов после чисел
        text = re.sub(r'(\d+)\s*процент', r'\1 процент', text)

        # Шаг 6: Обработка валют после чисел
        text = re.sub(r'(\d+)\s*(доллар|евро|рубль|фунт|иена)', r'\1 \2', text)

        # Шаг 7: Очистка лишних символов
        text = re.sub(r'[«»"“”„\[\]()]', '', text)

        # Шаг 8: Удаление лишних пробелов
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    def high_quality_speed_up(self, audio: np.ndarray, speed_factor: float) -> np.ndarray:
        """Качественное УСКОРЕНИЕ (только ускорение, без замедления!)"""
        if abs(speed_factor - 1.0) < 0.01 or speed_factor < 1.0:
            return audio  # Не замедляем!

        # Ограничиваем максимальное ускорение для сохранения качества
        if speed_factor > 2.0:
            print(f"  Внимание: большое ускорение {speed_factor:.2f}x, ограничиваем до 2.0x")
            speed_factor = 2.0

        # Параметры для лучшего качества
        n_fft = 2048
        hop_length = n_fft // 4

        try:
            # STFT преобразование
            stft = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length, window='hann')

            # Phase vocoder для ускорения
            stft_stretched = librosa.phase_vocoder(stft, rate=speed_factor, hop_length=hop_length)

            # Обратное преобразование
            audio_stretched = librosa.istft(stft_stretched, hop_length=hop_length, window='hann')

            return audio_stretched.astype(np.float32)

        except Exception as e:
            print(f"  Ошибка при ускорении: {e}")
            # Резервный метод - обрезка
            target_samples = int(len(audio) / speed_factor)
            return audio[:target_samples]

    def apply_smooth_fade(self, audio: np.ndarray, fade_in_ms: int = 5, fade_out_ms: int = 10) -> np.ndarray:
        """Применяет плавное затухание для предотвращения щелчков"""
        if len(audio) < 100:  # Слишком короткий для фейдов
            return audio

        # Fade-in в начале
        fade_in_samples = int(fade_in_ms * self.sample_rate / 1000)
        if fade_in_samples > 0 and len(audio) > fade_in_samples:
            fade_in = np.linspace(0, 1, fade_in_samples)
            audio[:fade_in_samples] *= fade_in

        # Fade-out в конце
        fade_out_samples = int(fade_out_ms * self.sample_rate / 1000)
        if fade_out_samples > 0 and len(audio) > fade_out_samples:
            fade_out = np.linspace(1, 0, fade_out_samples)
            audio[-fade_out_samples:] *= fade_out

        return audio

    def trim_excessive_silence(self, audio: np.ndarray, threshold_db: float = -30) -> np.ndarray:
        """Убирает слишком длинную тишину в начале и конце"""
        if len(audio) == 0:
            return audio

        # Преобразуем в dB
        audio_db = librosa.amplitude_to_db(np.abs(audio), ref=np.max)

        # Находим ненулевые участки (где громкость выше порога)
        above_threshold = audio_db > threshold_db

        if not np.any(above_threshold):
            return audio[:int(0.1 * self.sample_rate)]  # Возвращаем короткую тишину

        # Находим начало и конец речи
        speech_start = np.argmax(above_threshold)
        speech_end = len(audio) - np.argmax(above_threshold[::-1])

        # Оставляем небольшие паузы (50мс) в начале и конце
        padding = int(0.05 * self.sample_rate)
        start = max(0, speech_start - padding)
        end = min(len(audio), speech_end + padding)

        return audio[start:end]

    def generate_audio_segment(self, text: str, target_duration: float) -> np.ndarray:
        # Записываем оригинальный текст для логирования
        original_text = text

        # Предобработка текста
        text = self.preprocess_text(text.strip())

        if not text or len(text) < 2:
            print(f"  Пустой текст, возвращаем тишину")
            return np.zeros(int(target_duration * self.sample_rate), dtype=np.float32)

        # Логирование преобразования
        print(f"  Оригинал: {original_text[:80]}...")
        print(f"  Обработан: {text[:80]}...")

        # Ограничение длины текста для стабильности
        if len(text) > 350:
            print(f"  Текст слишком длинный ({len(text)} символов), обрезаем")
            text = text[:340] + "..."

        try:
            # Генерация аудио
            with torch.no_grad():
                wav = self.model.apply_tts(
                    text=text,
                    speaker='xenia',
                    sample_rate=self.sample_rate,
                    put_accent=True,
                    put_yo=True
                )

            audio = wav.cpu().numpy().squeeze()

            # Убираем лишнюю тишину по краям
            audio = self.trim_excessive_silence(audio)

            # Нормализация
            if np.max(np.abs(audio)) > 0:
                audio = audio / np.max(np.abs(audio)) * 0.95

            # Текущая длительность
            current_duration = len(audio) / self.sample_rate
            print(f"  Длительность сгенерированного: {current_duration:.2f}s")
            print(f"  Целевая длительность: {target_duration:.2f}s")

            # РЕШАЕМ: что делать с длительностью?
            if current_duration > target_duration:
                # Случай 1: русский текст ДЛИННЕЕ - УСКОРЯЕМ
                speed_factor = current_duration / target_duration
                print(f"  Русский текст длиннее, ускоряем в {speed_factor:.2f} раз")
                audio = self.high_quality_speed_up(audio, speed_factor)
                current_duration = len(audio) / self.sample_rate
                print(f"  Длительность после ускорения: {current_duration:.2f}s")

            # Финальная коррекция длины
            target_samples = int(target_duration * self.sample_rate)

            if len(audio) > target_samples:
                # Если после ускорения все еще длиннее - обрезаем с fade-out
                excess_samples = len(audio) - target_samples
                print(f"  После ускорения все еще длиннее, обрезаем {excess_samples} сэмплов")

                # Применяем fade-out перед обрезкой
                fade_out_samples = min(100, excess_samples)
                if fade_out_samples > 0:
                    audio[target_samples - fade_out_samples:target_samples] *= np.linspace(1, 0, fade_out_samples)

                audio = audio[:target_samples]

            elif len(audio) < target_samples:
                # Случай 2: русский текст КОРОЧЕ - ПРОСТО ДОБАВЛЯЕМ ТИШИНУ
                silence_samples = target_samples - len(audio)
                silence_duration = silence_samples / self.sample_rate
                print(f"  Русский текст короче, добавляем {silence_duration:.2f}s тишины")

                # Просто дополняем тишиной в конце
                audio = np.pad(audio, (0, silence_samples), mode='constant')

            else:
                # Идеально подошло по времени
                print(f"  Идеальная длительность! ±0.01s")

            # Применяем плавные переходы
            audio = self.apply_smooth_fade(audio, fade_in_ms=5, fade_out_ms=10)

            # Финальная проверка
            final_duration = len(audio) / self.sample_rate
            duration_diff = abs(final_duration - target_duration)

            if duration_diff > 0.01:
                print(f"  Внимание: итоговая длительность отличается на {duration_diff:.3f}s")
            else:
                print(f"  Итоговая длительность: {final_duration:.2f}s (цель: {target_duration:.2f}s)")

            return audio.astype(np.float32)

        except Exception as e:
            print(f"  Ошибка при генерации аудио: {e}")
            return np.zeros(int(target_duration * self.sample_rate), dtype=np.float32)

In [ ]:
# 6. Создание финального аудио
def create_final_audio(translated_segments, tts, original_audio_duration):
    print("Создаем финальное аудио...")

    # Создаем массив для финального аудио
    final_audio = np.zeros(int(original_audio_duration * tts.sample_rate), dtype=np.float32)
    processed_segments = 0

    for i, segment in enumerate(translated_segments):
        print(f"Синтезируем сегмент {i+1}/{len(translated_segments)}: '{segment['translated_text'][:50]}...'")

        try:
            synthesized_audio = tts.generate_audio_segment(
                segment['translated_text'],
                segment['duration']
            )

            # Проверяем что синтезированное аудио валидно
            if synthesized_audio is None or len(synthesized_audio) == 0:
                print(f"Пустое аудио для сегмента {i+1}, пропускаем")
                continue

            # Рассчитываем позицию в финальном аудио
            start_sample = int(segment['start'] * tts.sample_rate)
            end_sample = start_sample + len(synthesized_audio)

            # Проверяем границы
            if start_sample >= len(final_audio):
                print(f"Сегмент {i+1} выходит за границы, пропускаем")
                continue

            if end_sample > len(final_audio):
                print(f"Сегмент {i+1} выходит за границы, обрезаем")
                synthesized_audio = synthesized_audio[:len(final_audio) - start_sample]
                end_sample = len(final_audio)

            # Добавляем в финальное аудио
            final_audio[start_sample:end_sample] = synthesized_audio
            processed_segments += 1

        except Exception as e:
            print(f"Ошибка при обработке сегмента {i+1}: {e}")
            continue

    print(f"Успешно обработано {processed_segments}/{len(translated_segments)} сегментов")
    return final_audio

In [ ]:
# 7. Основной процесс
def process_video_file():
    # Загружаем видеофайл
    video_path = upload_video_file()
    if not video_path:
        print("Файл не загружен")
        return

    # Извлекаем аудио
    audio_path = extract_audio_from_video(video_path)
    if not audio_path or not os.path.exists(audio_path):
        print("Не удалось извлечь аудио")
        return

    # Получаем длительность оригинального аудио
    try:
        original_audio, orig_sr = librosa.load(audio_path, sr=None)
        original_duration = len(original_audio) / orig_sr
        print(f"Длительность аудио: {original_duration:.2f} секунд")
    except Exception as e:
        print(f"Ошибка загрузки аудио: {e}")
        return

    # Транскрибируем
    segments = transcribe_with_timestamps(audio_path)
    if not segments:
        print("Не удалось транскрибировать аудио")
        return

    # Переводим
    translator = Translator()
    translated_segments = translator.translate_segments(segments)
    if not translated_segments:
        print("Не удалось перевести сегменты")
        return

    # Синтезируем речь
    tts = SileroTTSWithTiming()
    final_audio = create_final_audio(translated_segments, tts, original_duration)

    # Сохраняем результат
    output_path = 'translated_audio.wav'
    sf.write(output_path, final_audio, tts.sample_rate)

    print(f"Готово! Результат сохранен в {output_path}")
    print(f"Размер файла: {os.path.getsize(output_path) / (1024*1024):.2f} MB")

    # Предлагаем скачать результат
    print("Скачиваем результат...")
    files.download(output_path)

    return output_path

# Запускаем процесс
print("Запускаем процесс обработки видео...")
result = process_video_file()

if result:
    print("🎉 Процесс завершен успешно!")
else:
    print("❌ Процесс завершен с ошибками")

In [ ]:
# Пробный тест синтеза речи
from IPython.display import Audio, display

model_id = 'v5_cis_base'
model, example_text = silero_tts(language='ru',
                                 speaker=model_id)
model.to(device)

sample_rate = 48000
speaker = 'ru_alexandr'
'''
доступные голоса

should be in bak_aigul, ru_aigul, tat_albina, ru_albina, erz_alexandr,
ru_alexandr, bak_alfia, ru_alfia, bak_alfia2, ru_alfia2, bel_anatoliy,
udm_bogdan, ru_bogdan, bel_dmitriy, ru_dmitriy, chv_ekaterina, ru_ekaterina,
kat_vika, ru_vika, aze_gamat, ru_gamat, ukr_igor, ru_igor, kjh_karina,
ru_karina, xal_kejilgan, ru_kejilgan, xal_kermen, ru_kermen, bel_larisa,
tat_marat, ru_marat, bak_miyau, ru_miyau, kir_nurgul, ru_nurgul, mdf_oksana,
ru_oksana, tgk_onaoy, ru_onaoy, bak_ramilia, ru_ramilia, ukr_roman, ru_roman,
tgk_safarhuja, ru_safarhuja, uzb_saida, ru_saida, kjh_sibday, ru_sibday,
hye_zara, ru_zara, kaz_zhadyra, ru_zhadyra, kaz_zhazira, ru_zhazira,
sah_zinaida, ru_zinaida, ru_eduard, kbd_eduard
'''

example_text = 'брод+ить с дождём под окнами твоими.'

audio = model.apply_tts(text=example_text,
                        speaker=speaker,
                        sample_rate=sample_rate)
print(example_text)
display(Audio(audio, rate=sample_rate))